# HMM（隐马尔可夫模型）

HMM是统计模型，它用来描述一个含有隐含未知参数的马尔可夫过程。其难点是从可观察的参数中确定
该过程的隐含参数。

HMM 解决三个问题，一个是隐藏序列的概率，一个是求状态转移矩阵，一个是给定观察序列求隐藏序列

直接上例子更清晰：http://www.cnblogs.com/skyme/p/4651331.html参考了大拿的文章。

假设手里有三个骰子，第一个骰子是正方体（称为D6）,每个面（1,2,3,4,5,6），出现等概率1/6 ,第二个骰子是个
四面体（D4）,每个面等概率出现1/4,第三个骰子为八面体（1,2,3,4,5，6,7,8）等概率出现1/8
![](./pics/HMM1.png)

假设我们从三个骰子里挑一个，概率都是1/3,然后掷骰子，得到一个数字，1,2,3,4,5,6,7,8中的一个，不停的重复上述过程，我们会得到一串数字
（掷10次）：1, 6 3 5 7 3 5 2 4

这串数字叫做可见状态链（observation也称可观察的），但是还有一串隐含状态链，在这个例子中这串隐含状态链就是你用的骰子的序列，比如：
隐含状态链有可能是：D6 D8 D8 D6 D4 D8 D6 D6 D4 D8

一般来说HMM中的马尔可夫链其实是指隐含状态链，所以称为隐马尔可夫链（哈哈），因为隐含状态之间存在转换概率（transition probability）
在这个例子里，D6的下一个状态是D4，D6,D8的概率都是1/3,D4，D4，D8也一样，互相之间转换概率都是1/3,这样设定是为了
最开始容易理解，但是实际上我们可以随意设置转换概率，比如我们可以这样定义：D6后面不能接D4,D6，后面是D6的概率是0.9
是D8的概率是0.1,这样就是一个新的HMM.

同样的，尽管可见状态之间没有转换概率，但是隐含状态和可见状态之间有一个概率叫做输出概率（emission probability）
就这个例子来讲，六面骰产生1的输出概率是1/6,产生1,6都是1/6.同样我们可以对输出概率进行设定，比如骰子的某些位置密度更大，导致骰子掷出
1的概率更大是1/2,而2,3,4,5,6，的概率是1/10.
![](./pics/HMM2.png)![](./pics/HMM3.png)

其实对于HMM来说，如果提前知道所有隐含状态之间的转换概率和所有隐含状态到所有可见状态之间的输出概率，模拟是很容易的
但是往往应用HMM模型时缺失了一部分信息，有时候知道骰子有几种，每种骰子是什么但是不知道掷出来的序列；有时候只看到了很多次掷骰子的结果，剩下的什么都不知道。
如何应用算法去估计这些缺失的信息，就是一个很重要的问题。
HMM模型相关的算法主要分三类，分别解决三种问题：

- 1、知道隐含状态数量和转换概率，根据可见状态链，求出隐含状态链，即（知道骰子有几种每种骰子是什么根据掷出来的结果，推出每次掷出的是哪个骰子）说白了就是求序列。

这个问题在语音识别领域，叫解码问题，这个问题有两个不同的答案。每个答案都对只是意义不一样。第一种解法是：
求最大似然路径，说人话就是我求一串骰子序列，这串骰子序列产生观测结果的概率最大。
第二种解法：就不是求一组骰子序列了，而是求每次掷出来的骰子分别是某种骰子的概率，比如说
我看到结果后，我可以求得第一次掷骰子是D4的概率是0.5，D6的概率是0.3，D8的概率是0.2，下面会将第一种解法

- 2、还是知道隐含状态数量和转换概率，根据可见状态链，想知道掷出这个结果的概率（即知道骰子有几种和每个骰子是什么以及掷出来的结果，求结果的概率）

看似这个问题意义不大，因为你掷出来的结果很多时候都对应了一个比较大的概率，这个问题的目的是检测观察到的结果是否和模型是否吻合，如果很多次结果都对应了比较小的概率，那就说明已知模型可能是错的，有人偷偷把骰子换了。

- 3、还是知道隐含状态数量，不知道转换概率，但是知道可见状态链，反推转换概率

这个问题很重要，因为这是最场景的问题，很多时候我们只知道结果，不知道HMM模型里的参数，我们需要
从可见结果估计出这些参数，这是建模的一个必要步骤。

下面开始说解法

0 、 一个简单的问题

只是作为解决上述问题的一个辅助，对上面的问题有帮助
知道骰子有几种，每种骰子是什么，每次掷的都是什么骰子，根据骰子掷出来的结果，求产生这个骰子的结果
![](./pics/HMM4.png)

解法就是每步概率相乘：
P = 转换概率*输出概率
$$
\begin{aligned}
P =& P(D6) * P(D6->1) *P(D6->D8)*P(D8->6)*P(D8->D8)*P(D8->3)\\
=&\frac{1}{3}\times \frac{1}{6}\times \frac{1}{3}\times \frac{1}{8}\times \frac{1}{3}\times \frac{1}{8}
\end{aligned}
$$

#### 1 、看见不可见的，破解骰子序列

这里讲的是第一种解法：最大似然路径问题。

举例来说，知道三个骰子，也知道掷了10次骰子的结果，不知道每次用了哪种骰子，想知道最有可能的骰子序列
这是不是有点想知道有n条路通向罗马，但当你的朋友已经到达罗马时，你想知道他是通过其中哪几条到达的一样，这似乎可以理解为最优化路径问题，其实最简答暴力的方法就是穷举所有可能的骰子的序列，然后按照第0个问题的方式计算出每个序列的概率，概率最大的就是最可能的序列，但是如果马尔可夫链太长的话计算量就太大了。

于是另外一种方法诞生了，叫做Viterbi 算法：要理解这个算法，先来看几个简单的例子：

首先,如果我们只掷一次骰子，看到结果为1，对应的最大概率骰子序列就是D4,因为它只有4面啊1/4的概率高于1/6,1/8，
扩展一下，我们掷两次骰子，结果为1,6，这是问题变得复杂了，我们要计算三个值，分别是第二个骰子是D4,D6，D8的最大概率，
![](./pics/HMM5.png)
要取到最大概率，第一个骰子必须为D4,这时第二个骰子取到D6的最大概率是：
![](./pics/HMM6.png)
$$
\begin{aligned}
P2(D6) =& P(D4) * P(D4->1) *P(D4->D6)*P(D6->6)\\
=&\frac{1}{3}\times \frac{1}{4}\times \frac{1}{3}\times \frac{1}{6}
\end{aligned}
$$

同样，我们计算第二个骰子是D4或D8的最大概率，可以看出第二个骰子取到D6的概率最大，而使这个概率最大时，第一个骰子为D4,所以最大概率骰子序列是D4,D6.

继续扩展,掷三次骰子：
![](./pics/HMM7.png)

同样我们计算第三个骰子分别是D4,D6,D8的最大概率，要取到最大概率，第二个骰子必须是D6,这时取到D4的最大概率是
$$
\begin{aligned}
P3(D4) =& P2(D6) * P(D6->D4) *P(D4->3)\\
=&\frac{1}{216}\times \frac{1}{3}\times \frac{1}{4} 
\end{aligned}
$$

同上，可以计算出第三个骰子是D6,或D8时的最大概率，可以发现第三个骰子取到D4的概率最大，而使这个概率最大时，第二个骰子是D6,第一个骰子是D4.所以最大骰子序列就是D4，D6,D4.

总结规律：既然掷一二三次可以算，掷多少次都可以以此类推，我们发现，要求最大概率筛子序列时要做着几件事：

- 不管序列多长，要从序列长度为1算起，序列长度为1时取到每个骰子的最大概率。
- 逐渐增加长度，每增加一次长度，重新算一遍在这个长度下最后一个位置取到每个骰子的最大概率，因为上一个长度下的取到每个骰子的最大概率都算过了，重新计算的话其实不难，当我们算到最后一位时，就知道最后一位时哪个骰子的概率最大了。
- 最后把对应这个最大概率的序列从后往前推出来。

#### 2、谁动了我的骰子

比如说你怀疑自己的六面骰子被动了手脚，有可能被换成了另一种六面骰，这种六面骰掷出来是1的概率更大是1/2，其他的是1/10,你怎么办，答案很简单，算一算正常的三个骰子
掷出来一段序列的概率，再算一算不正常的六面骰和另外两个正常骰子掷出这段序列的概率，如果前者比后者小，
就要小心了。

In [ ]:
要算用正常的三个骰子掷出这个结果的概率，其实就是将所有可能情况的概率进行加和计算，同样，
简单而暴力的方法就是穷举所有的骰子序列，还是计算每个骰子序列对应的概率，但是这回，我们不挑最大值了
而是把所有算出来的概率相加，得到总概率就是我们要求的结果，这个方法依然不能应用于太长的骰子序列
（马尔科夫链）

我们会应用一个和前一个问题类似的解法，只不过前一个问题率最大值，这个问题关心的是概率之和
解决这个问题的算法叫做前向算法
首先我们只掷一次骰子：看到结果为1，产生这个结果的总概率可以按照如下计算，总概率为0.18
![](./pics/H2-0.png)
![](./pics/H2.png)

把这个情况扩展，掷两次骰子：
![](./pics/H2-2.png)
看到结果为1,6，产生这个结果的总概率可以按照如下计算，总概率为0.05
![](./pics/H2-3.png)

继续扩展为3次：
![](./pics/H2-4.png)
看到结果为1,6,3产生这个结果的概率可以按照如下计算，总概率为0.03
![](./pics/H2-5.png)

同样的，我们一步一步的算，有多长算多长，再长的马尔科夫链总能算出来，用同样的方法，也可以算出
不正常的六面骰和另外两个正常骰掷出这段序列的概率，然后我们比较一下这两个概率大小，就能知道骰子是不是被换了

### Viterbi 算法

Viterbi被广泛应用到分词，词性标注等应用场景。

HMM是用来描述隐含未知参数的统计模型：举个例子，一个东京的朋友每天根据天气{下雨，天晴}决定
当天的活动{公园散步，购物，清理房间}中的一种，我每天只能在twitter上看到他的状态，"我前天公园散步，昨天购物，今天清理房间"那么我可以根据
他发的推断东京这三天的天气，在这个例子里，显状态是活动，隐状态是天气。

任何一个HMM都可以用一个五元组来表示：

：param observation:观测序列
：param states: 隐状态
：param start_probability :初始概率（隐状态）
：param trans_probability :转移概率（隐状态）
：param emit_probability：发射概率（输出概率，也是隐状态表现为显状态的概率）
![](./pics/Viterbi.jpg)

In [8]:
states = {'Rainy','Sunny'}
observations = {'walk','shop','clean'}
start_probability = {'Rainy':0.6,'Sunny':0.4}
transition_probability = {
    
    'Rainy':{'Rainy':0.7,'Sunny':0.3},
    'Sunny':{'Rainy':0.4,'Sunny':0.6},
}
emission_probability = {
    
    'Rainy':{'walk':0.1,'shop':0.4,'clean':0.5},
    'Sunny':{'walk':0.6,'shop':0.3,'clean':0.1},
}

### 求解最可能的天气

求解最可能的隐状态序列是HMM的三个典型问题之一，通常用维特比算法解决，它是求解HMM上的最短路径（-log(prob)，也即是最大概率）的算法
思路：

- 1、定义V[时间][今天天气] = 概率，注意今天天气指的是，前几天的天气都确定下来了（概率最大）今天天气是X的概率，
这里的概率就是一个累乘的概率
- 2、因为第一天我朋友去散步了，可能是下雨天也可能是晴天，下雨的概率V[第一天][下雨] = 初始概率[下雨] * 发射概率[下雨][散步] = 0.6*01. = 0.05
同理V[第一天][晴天] = 初始概率[晴天]*发射概率[晴天][散步] = 0.6*0.4=0.24，从直觉上看，因为她第一天出门了，她一般喜欢在天晴的时候散步
很符合人的习惯，所以第一天天晴的概率比较大。
- 3、从第二天开始对于每种天气Y，都有前一天天气是X的概率 * X转移到Y的概率*Y天气下某活动的概率。因为
前一天天气X有两种可能，所以Y的概率有两个，选取其中较大的一个作为V[第二天][天气Y]的概率，同时将今天的天气加入到结果序列中。
- 4、比较V[最后一天][下雨]和[最后一天][天晴]的概率，找出较大的那一个对应的序列，就是最终结果。


In [12]:
#-*- coding:utf-8 -*-
#Filename:viterbi.py
#Author:hcc
#Data:2018-09-02

#Day 0:计算Sunny、Rainy的概率
#观察到的是walk，
#V[0][Sunny] = start_p[Sunny]*emit_p[Sunny][walk]
#V[0][Rainy] = start_p[Rainy]*emit_p[Rainy][walk]
#path[y]=[y]
# 

#Day 1:计算Sunny，Rainy的概率
#观察到的是shop
#newpath = {}
#V[1][Sunny] = V[0][Sunny] *tran_p[V[0][Sunny]][Sunny] *emit_p[Sunny][shop]
#V[1][Sunny] = V[0][Sunny] *tran_p[V[0][Sunny]][Rainy] *emit_p[Sunny][shop]
#取最大的得到Pro，PreState ,curState,更新newpath[y] = path[PreState]+[curState]
#V[1][Rainy] = V[0][Rainy] *tran_p[V[0][Rainy]][Sunny] *emit_p[Rainy][shop]
#V[1][Rainy] = V[0][Rainy] *tran_p[V[0][Rainy]][Rainy] *emit_p[Rainy][shop]
#取最大的得到Pro，PreState ,curState,更新newpath[y] = path[PreState]+[curState]
#path = newPath

#Day 2:计算Sunny、Rainy的概率
#观察到的是clean
#V[2][Sunny] = V[1][Sunny]* tran_p[V[1][Sunny]][Sunny] *emit_p[Sunny][clean]
#V[2][Sunny] = V[1][Sunny]* tran_p[V[1][Sunny]][Rainy] *emit_p[Sunny][clean]
#取最大的得到Pro，PreState ,curState,更新newpath[y] = path[PreState]+[curState]
#V[2][Rainy] = V[1][Rainy]* tran_p[V[1][Rainy]][Sunny] *emit_p[Rainy][clean]
#V[2][Rainy] = V[1][Rainy]* tran_p[V[1][Rainy]][Rainy] *emit_p[Rainy][clean]
#path = newPath
#最后一个取概率最大pro对应的state，序列即path[state]
 
def print_dptable(V):
    
    for i in range(len(V)): print("\t%12d" % i,end='')
    print()
    for y in V[0].keys():
        print()
        print("%7s: " % y,end='')
        for t in range(len(V)):
            print ("\t%.7s\t" % ("%f" % V[t][y]),end='')
        print()
    print("\n")
def viterbi(obs,states,start_p,trans_p,emit_p):
    
    V = [{}]
    path = {}
    #初始化初始状态t=0
    for y in states:
        V[0][y] = start_p[y] * emit_p[y][obs[0]]
        path[y] = [y]
    #t>0时刻之后   
    for t in range(1,len(obs)):
        V.append({})
        newPath ={}
        for y1 in states:
            #隐状态 = 前状态是y0的概率 * y0转移到y的概率 * y表现为当前状态的概率
            (MaxPro,PreState) = max([(V[t-1][y0]*trans_p[y0][y1] * emit_p[y1][obs[t]],y0) for y0 in states])
            V[t][y1] = MaxPro
            newPath[y1] = path[PreState] + [y1] #从前向后的路径都不断的更新，分两条线更新：分别是初始状态为Rainy和Sunny，朝着概率最大的方向
        path = newPath
    
    (prob,state) = max([(V[len(obs)-1][y],y) for y in states])
    print_dptable(V)
    return (prob,path[state])
def main():
    states = ('Rainy','Sunny')
    obs = ('walk','shop','clean')
    start_p = {'Rainy':0.6,'Sunny':0.4}
    trans_p = {
        
        'Rainy':{'Rainy':0.7,'Sunny':0.3},
        'Sunny':{'Rainy':0.4,'Sunny':0.6},
    }
    emit_p = {
        
        'Rainy':{'walk':0.1,'shop':0.4,'clean':0.5},
        'Sunny':{'walk':0.6,'shop':0.3,'clean':0.1},
    }    
    return viterbi(obs, states, start_p, trans_p, emit_p)
if __name__ =="__main__":
    pro,seq = main()
    print("Most Prob Seq:\n")
    print(pro,seq)

	           0	           1	           2

  Rainy: 	0.06000		0.03840		0.01344	

  Sunny: 	0.24000		0.04320		0.00259	


Most Prob Seq:

0.01344 ['Sunny', 'Rainy', 'Rainy']
